# 4. 모델 훈련

## 4.1 선형 회귀
선형 모델은 아래 식처럼 입력 특성의 가중치 합와 편향<small>bias</small>이라는 상수를 이용해 만든다.

$$ \widehat{y}=\theta_0 + \theta_1x_1 + \theta_2x_2 + \cdots + \theta_nx_n $$

- $y$는 예측값이다.
- $n$은 특성의 수이다.
- $x_i$는 $i$번째 특성의 값이다.
- $\theta_i$는 $i$번째 모델 파라미터이다. 편향 $\theta_0$과 특성의 가중치 $\theta_1, \theta_2, \cdots, \theta_n$이 있다.

이를 간단히 아래처럼 만들 수 있다.

$$ \widehat{y} = h_\theta(X) = \theta \cdot X $$

모델을 훈련시킨다는 것은 모델이 훈련 세트에 맞도록 파라미터를 변경한다는 것이다.
이를 위해서는 모델이 훈련 데이터에 얼마나 잘 맞는지 평가해야 한다.
성능 측정 지표 중 하나인 평균 제곱 오차(MSE)는 다음과 같다.

$$ \mathrm{MSE}(X, h_\theta)=\frac{1}{m} \sum_{i=1}^{m} (\theta^T\mathrm{x}^{(i)}-y^{(i)})^2 $$

### 4.1.1 정규방정식